### 라이브러리 선언

In [35]:
import pandas as pd
import numpy as np

### 데이터 불러오기

In [36]:
dataUrl = "https://raw.githubusercontent.com/hyokwan/python-lecture/master/dataset/kopo_channel_seasonality_new.csv"

In [37]:
selloutData = pd.read_csv(dataUrl)

# A. 데이터 전처리

### 1. 데이터 통합 및 불량데이터 처리

In [38]:
## regionid : str, product: str, yearweek: int 
## qty: float

In [39]:
selloutData.REGIONID = selloutData.REGIONID.astype(str)
selloutData.PRODUCT = selloutData.PRODUCT.astype(str)
selloutData.YEARWEEK = selloutData.YEARWEEK.astype(int)
selloutData.QTY = selloutData.QTY.astype(float)

In [40]:
selloutData["YEAR"] = selloutData.YEARWEEK.astype(str).str[:4]
selloutData["WEEK"] = selloutData.YEARWEEK.astype(str).str[4:]

[불량 데이터 처리]
kopo_channel_seasonality_new.csv 자료를 담은 
selloutData 변수에서
QTY컬럼 음수(반품)인 경우 0, 양수인 경우 기존 QTY 값유지하는 로직을 적용하여 QTY_NEW 컬럼을 추가하세요

In [41]:
selloutData["QTY_NEW"] = np.where( selloutData.QTY < 0, 0, selloutData.QTY )

### 2. 데이터 통합

[데이터 통합]
selloutData 자료에서 
YEAR, WEEK 컬럼을 생성하고 WEEK 가 52 이하인
데이터만 조회한 후 refinedSelloutData 변수에 담으세요


In [42]:
exceptWeek = "53"
refinedSelloutData = selloutData.loc[  selloutData.WEEK != exceptWeek    ]

refinedSelloutData 에서 →
지역, 상품, 연주차 컬럼순으로 오름차순 정렬하여
sortedData 변수에 담으세요


# B. 추세선 도출

In [22]:
sortKey = ["REGIONID","PRODUCT","YEARWEEK"]

sortedData = refinedSelloutData.sort_values(by=sortKey)

[지역, 상품, 연도 별 집계]
sortedData 에서 지역, 상품, 연도 단위
판매량(QTY_NEW) 의 평균 연산 후
groupData 변수에 담으세요

이후 컬럼명을 QTY_MEAN로 변경하세요


In [99]:
groupKey = ["REGIONID","PRODUCT","YEAR"]
groupData = sortedData.groupby(by=groupKey)[["QTY_NEW"]].agg("mean").reset_index()
groupData = groupData.rename(columns={"QTY_NEW":"QTY_MEAN"})

refinedSelloutData와 groupData를 
[REGIONID, PRODUCT, YEAR] 키로 조인하여 
mergedData 변수에 아래와 같이 담으세요

In [100]:
joinKey = ["REGIONID","PRODUCT"]

In [101]:
mergedData = pd.merge(left=refinedSelloutData,
         right=groupData,
         on = joinKey,
         how = "left")

In [102]:
mergedData = mergedData.sort_values(by=sortKey)

# C. 계절성지수 산출

계절성 지수는
QTY_NEW / QTY_MEAN
으로 산출하여 SEASONALITY 컬럼을 생성한다.


In [103]:
mergedData["SEASONALITY"] = mergedData.QTY_NEW / mergedData.QTY_MEAN

마지막으로 [지역, 상품, 주차] 별 
계절성 지수 평균 값을
조별 산출하여 finalResult 변수에 담으세요


In [104]:
groupKey = ["REGIONID","PRODUCT","WEEK"]

In [105]:
seasonData = mergedData.groupby(groupKey)[["SEASONALITY"]].agg("mean").reset_index()

In [107]:
seasonData

,REGIONID,PRODUCT,WEEK,SEASONALITY
0,A00,PRODUCT34,01,2.086814
1,A00,PRODUCT34,02,2.234620
2,A00,PRODUCT34,03,1.773667
3,A00,PRODUCT34,04,1.788698
4,A00,PRODUCT34,05,1.513128
...,...,...,...,...
41283,A77,PRODUCT12,48,1.208670
41284,A77,PRODUCT12,49,1.098241
41285,A77,PRODUCT12,50,1.313242
41286,A77,PRODUCT12,51,1.159889


In [97]:
pd.DataFrame( [["A00","PRODUCT34","202401",0]],
            columns = ["REGIONID","PRODUCT","WEEK","SEASONHK"])

,REGIONID,PRODUCT,WEEK,SEASONHK
0,A00,PRODUCT34,202401,0
